In [3]:
import json
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

# ==========================================
# 1. SETUP THE SCORING PIPELINE
# ==========================================

# A. Define the Evaluation Prompt
# We instruct the LLM to ignore technical BPMN artifacts (like "missing flow") 
# and focus on the linguistic quality of the description.
readability_template = """
You are an expert linguist and communication coach. 
Evaluate the following text which describes a business process.

Criteria for Scoring (0-10):
- **10 (Perfect):** Flows naturally like a story, clear transitions, no ambiguity.
- **7-9 (Good):** Clear, but slightly repetitive or formal.
- **4-6 (Average):** Understandable, but robotic or contains minor errors.
- **0-3 (Poor):** Confusing, disjointed, or grammatically incorrect.

Text to Evaluate:
"{{ passage }}"

Instructions:
1. Ignore technical notes like "(missing Description message flow)".
2. Assign a single integer score from 0 to 10.
3. Return the result strictly as a valid JSON object.
Example format: {"score": 8, "reason": "Good flow but slightly passive voice."}
"""

# B. Initialize Components
prompt_builder = ChatPromptBuilder(
    template=[ChatMessage.from_user(readability_template)],
    required_variables=["passage"]
)

# Initialize Ollama (Using Llama 3.1:8b)
llm = OllamaChatGenerator(
    model="llama3.1:8b", 
    url="http://localhost:11434",
    generation_kwargs={
        "format": "json",    # Force JSON output
        "temperature": 0.0   # Deterministic (consistent scores)
    }
)

# C. Build Pipeline
readability_pipeline = Pipeline()
readability_pipeline.add_component("prompt_builder", prompt_builder)
readability_pipeline.add_component("llm", llm)
readability_pipeline.connect("prompt_builder.prompt", "llm.messages")

# ==========================================
# 2. INPUT DATA
# ==========================================

# The specific text you provided
bpmn_description = """
The process is initiated when the passenger shows their boarding pass. Afterwards, the passenger proceeds to the security check area to undergo a security check. Following the screening, a decision is made regarding whether the passenger appears suspicious (missing Description message flow). If the passenger is deemed suspicious, they must undergo a manual control; if not, the manual control is skipped.
Subsequently, the process evaluates the passenger's destination via a gateway checking if the target destination is within the Schengen Area. If the destination is outside the Schengen Area, the passenger goes to passport control and shows their passport (missing Description message flow). Conversely, if the destination is within the Schengen Area, the passport control steps are bypassed. Finally, the passenger goes to the gate, concluding the process.
"""

# ==========================================
# 3. EXECUTE PIPELINE
# ==========================================

print("--- Evaluating Readability ---\n")

try:
    # Run the pipeline
    result = readability_pipeline.run({"prompt_builder": {"passage": bpmn_description}})
    
    # Extract the response text
    response_text = result["llm"]["replies"][0].text
    
    # Parse JSON
    data = json.loads(response_text)
    score = data.get("score", "N/A")
    reason = data.get("reason", "No reason provided")
    
    # Output
    print(f"BPMN Text Input: {bpmn_description[:60]}... (truncated)\n")
    print(f"✅ Readability Score: {score}/10")
    print(f"📝 Reasoning:        {reason}")

except Exception as e:
    print(f"Error executing pipeline: {e}")

--- Evaluating Readability ---

BPMN Text Input: 
The process is initiated when the passenger shows their boa... (truncated)

✅ Readability Score: 6/10
📝 Reasoning:        Understandable, but robotic and contains minor errors in sentence structure.


In [9]:
import xml.etree.ElementTree as ET
import json
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

# ==========================================
# 1. HELPER: PARSE BPMN XML
# ==========================================
def parse_bpmn_data(file_content):
    """
    Parses BPMN XML to extract:
    1. 'Human' text strictly from <bpmn:documentation> tags.
    2. 'Generated' text by listing task/gateway names sequentially.
    """
    try:
        root = ET.fromstring(file_content)
    except ET.ParseError as e:
        print(f"XML Parsing Error: {e}")
        return "", ""

    namespaces = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}
    
    # --- A. Extract Ground Truth (Documentation) ---
    doc_text = ""
    for doc in root.findall('.//bpmn:documentation', namespaces):
        if doc.text:
            doc_text += doc.text.strip() + " "
    
    if not doc_text:
        doc_text = "(No documentation tag found in file)"

    # --- B. Generate Robotic Text (Simulation) ---
    elements = []
    for elem in root.iter():
        # Clean tag name (remove {namespace})
        tag = elem.tag.split('}')[-1] 
        
        # Only look for flow nodes
        if tag in ['task', 'exclusiveGateway', 'startEvent', 'endEvent']:
            name = elem.get('name')
            if name:
                # Construct robotic sentence
                elements.append(f"{tag} executes '{name}'.")
    
    generated_text = " Next, ".join(elements)
    
    return doc_text.strip(), generated_text

# ==========================================
# 2. SETUP SCORING PIPELINE
# ==========================================
readability_template = """
You are a linguistic expert evaluating Business Process descriptions.

Criteria (0-10 Score):
- 10: Natural, cohesive, story-like flow (Human quality).
- 5: Factually correct but robotic, fragmented, or list-like.
- 0: Nonsensical or raw data dump.

Text to Evaluate: 
"{{ passage }}"

Return ONLY a JSON object: {"score": <int>, "reason": "<short explanation>"}
"""

# Define Prompt
prompt_builder = ChatPromptBuilder(
    template=[ChatMessage.from_user(readability_template)],
    required_variables=["passage"]
)

# Define Generator (Llama 3.1)
llm = OllamaChatGenerator(
    model="llama3.1:8b", 
    url="http://localhost:11434",
    generation_kwargs={"format": "json", "temperature": 0.0}
)

# Build Pipeline
pipeline = Pipeline()
pipeline.add_component("prompt", prompt_builder)
pipeline.add_component("llm", llm)
pipeline.connect("prompt.prompt", "llm.messages")

def score_text(text):
    """Runs the pipeline and returns the integer score."""
    if not text or text.startswith("(No"):
        return 0
    try:
        res = pipeline.run({"prompt": {"passage": text}})
        response_content = res["llm"]["replies"][0].text
        data = json.loads(response_content)
        return data.get("score", 0)
    except Exception as e:
        print(f"Scoring Error: {e}")
        return 0

# ==========================================
# 3. LOAD DATA (XML CONTENT)
# ==========================================

# [cite_start]1. passenger_security_cleaned.bpmn [cite: 1-13]
# [cite_start]Includes documentation [cite: 1-9]
xml_security_cleaned = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_1">
  <bpmn:process id="Process_0gjrx3e" isExecutable="true">
    <bpmn:documentation>The process is initiated when the passenger shows their boarding pass. Afterwards, the passenger goes to the security check area to undergo a security check. Following the screening, a decision is made regarding whether the passenger appears suspicious (missing Description message flow). If the passenger is deemed suspicious, they must undergo a manual control; if not, the manual control is skipped. Subsequently, the process evaluates the passenger's destination via a gateway so that the passenger is checked in if the target destination is in the Schengen Area. If the destination is outside the Schengen Area, the passenger goes to passport control and shows their passport (missing Description message flow). Conversely, if the destination is within the Schengen Area, the passport control steps are bypassed. Finally, the passenger goes to the gate, concluding the process.</bpmn:documentation>
    <bpmn:task name="Passenger shows boarding pass" />
    <bpmn:task name="Passenger goes to security check" />
    <bpmn:task name="Passenger is checked" />
    <bpmn:exclusiveGateway name="Suspicious?" />
    <bpmn:task name="manual control" />
    <bpmn:exclusiveGateway name="Target destination in Schengen-Area?" />
    <bpmn:task name="Passenger goes to passport control" />
    <bpmn:task name="Passenger shows passport" />
    <bpmn:task name="Passenger goes to gate" />
  </bpmn:process>
</bpmn:definitions>"""

# [cite_start]2. passenger_arrival.bpmn [cite: 14-44]
# [cite_start]Includes documentation [cite: 15-24]
xml_arrival = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_2">
  <bpmn:process id="Process_arrival" isExecutable="true">
    <bpmn:documentation>The process begins when the passenger arrives at the terminal. The flow immediately reaches an exclusive gateway that checks if the arrival is from the Schengen area. If the arrival is from the Schengen area, the passenger proceeds to passport control and shows their passport. If the arrival is not from the Schengen area, these control steps are skipped. Subsequently, the passenger goes to the baggage claim area. At this point, another gateway determines whether the passenger has baggage. If the passenger has baggage, they execute the task of taking their baggage. If they do not have baggage, they proceed directly to the next stage without taking baggage. Finally, the process reaches a decision point regarding customs. It checks if there are goods to declare. If there are goods to declare, the passenger goes to customs and declares them. If there are no goods to declare, the customs section is bypassed. The process concludes when the passenger leaves the airport.</bpmn:documentation>
    <bpmn:task name="Passenger arrives at terminal" />
    <bpmn:exclusiveGateway name="Arrival from Schengen area" />
    <bpmn:task name="Passenger goes to passport control" />
    <bpmn:task name="Passenger shows passport" />
    <bpmn:task name="Passenger goes to Baggage claim" />
    <bpmn:exclusiveGateway name="Passenger has Baggage?" />
    <bpmn:task name="Passenger takes baggage" />
    <bpmn:exclusiveGateway name="Goods to declare?" />
    <bpmn:task name="Passenger goes to customs" />
    <bpmn:task name="Passenger declares goods at customs" />
    <bpmn:task name="Passenger leaves airport" />
  </bpmn:process>
</bpmn:definitions>"""

# [cite_start]3. passenger_security.bpmn [cite: 45-59]
# The source file had NO documentation. 
# We INJECT the <bpmn:documentation> tag here so the extraction logic works.
xml_security = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_3">
  <bpmn:process id="Process_security" isExecutable="true">
    <bpmn:documentation>The process is initiated when the passenger shows their boarding pass. Afterwards, the passenger proceeds to the security check area to undergo a security check. Following the screening, a decision is made regarding whether the passenger appears suspicious. If the passenger is deemed suspicious, they must undergo a manual control; if not, the manual control is skipped. Subsequently, the process evaluates the passenger's destination via a gateway. If the destination is outside the Schengen Area, the passenger goes to passport control and shows their passport. Conversely, if the destination is within the Schengen Area, the passport control steps are bypassed. Finally, the passenger goes to the gate, concluding the process.</bpmn:documentation>
    <bpmn:task name="Passenger shows boarding pass" />
    <bpmn:task name="Passenger goes to security check" />
    <bpmn:task name="Passenger is checked" />
    <bpmn:exclusiveGateway name="Suspicious?" />
    <bpmn:task name="manual control" />
    <bpmn:exclusiveGateway name="Target destination in Schengen-Area?" />
    <bpmn:task name="Passenger goes to passport control" />
    <bpmn:task name="Passenger shows passport" />
    <bpmn:task name="Passenger goes to gate" />
  </bpmn:process>
</bpmn:definitions>"""

# ==========================================
# 4. RUN COMPARISON
# ==========================================
experiments = [
    {"name": "passenger_security_cleaned.bpmn", "xml": xml_security_cleaned},
    {"name": "passenger_arrival.bpmn", "xml": xml_arrival},
    {"name": "passenger_security.bpmn", "xml": xml_security}
]

print(f"{'BPMN MODEL':<35} | {'TYPE':<12} | {'SCORE':<5} | {'TEXT SAMPLE (Truncated)'}")
print("-" * 100)

for exp in experiments:
    # 1. Parse XML to get Ground Truth and Generate Robotic Text
    ground_truth, generated = parse_bpmn_data(exp["xml"])
    
    # 2. Score Both
    s_gt = score_text(ground_truth)
    s_gen = score_text(generated)
    
    # 3. Print Results
    print(f"{exp['name']:<35} | {'Ground Truth':<12} | {s_gt:<5} | {ground_truth[:40]}...")
    print(f"{'':<35} | {'Generated':<12} | {s_gen:<5} | {generated[:40]}...")
    print("-" * 100)

BPMN MODEL                          | TYPE         | SCORE | TEXT SAMPLE (Truncated)
----------------------------------------------------------------------------------------------------
passenger_security_cleaned.bpmn     | Ground Truth | 5     | The process is initiated when the passen...
                                    | Generated    | 5     | task executes 'Passenger shows boarding ...
----------------------------------------------------------------------------------------------------
passenger_arrival.bpmn              | Ground Truth | 5     | The process begins when the passenger ar...
                                    | Generated    | 5     | task executes 'Passenger arrives at term...
----------------------------------------------------------------------------------------------------
passenger_security.bpmn             | Ground Truth | 5     | The process is initiated when the passen...
                                    | Generated    | 5     | task executes 'Passenger s

In [11]:


import xml.etree.ElementTree as ET
import json
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

# ==========================================
# 1. METRIC: COHERENCE SCORING
# ==========================================
coherence_template = """
You are an expert text evaluator. Analyze the 'Coherence' of the following BPMN process description.

Criteria (0-10 Score):
- **10 (Excellent)**: Seamless flow, uses transition words (e.g., 'however', 'subsequently'), logical cause-and-effect structure.
- **5 (Average)**: Correct sequence but robotic, repetitive, or choppy sentences.
- **0 (Poor)**: Disjointed list of tasks with no narrative flow.

Text to Evaluate: 
"{{ passage }}"

Return ONLY a JSON object: {"score": <int>, "reason": "<short explanation>"}
"""

# Setup Pipeline
prompt_builder = ChatPromptBuilder(
    template=[ChatMessage.from_user(coherence_template)],
    required_variables=["passage"]
)

llm = OllamaChatGenerator(
    model="llama3.1:8b", 
    url="http://localhost:11434",
    generation_kwargs={"format": "json", "temperature": 0.0}
)

pipeline = Pipeline()
pipeline.add_component("prompt", prompt_builder)
pipeline.add_component("llm", llm)
pipeline.connect("prompt.prompt", "llm.messages")

def get_coherence_score(text):
    if not text: return 0, "No text provided"
    try:
        res = pipeline.run({"prompt": {"passage": text}})
        data = json.loads(res["llm"]["replies"][0].text)
        return data.get("score", 0), data.get("reason", "N/A")
    except:
        return 0, "Error parsing response"

# ==========================================
# 2. PARSING LOGIC
# ==========================================
def parse_bpmn_data(file_content):
    try:
        # Check if content is valid
        if not file_content.strip(): return "", ""
        root = ET.fromstring(file_content)
    except ET.ParseError: 
        return "Error: Invalid XML", ""
    
    # A. Extract Ground Truth (Human Text)
    # We look for documentation tags anywhere in the tree
    namespaces = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}
    doc_text = ""
    docs = root.findall('.//bpmn:documentation', namespaces)
    
    # If multiple docs exist (e.g. Collaboration vs Process), we pick the longest one
    if docs:
        doc_text = max([d.text for d in docs if d.text], key=len, default="")
    
    if not doc_text:
        doc_text = "(No documentation found in file)"

    # B. Generate Robotic Text (Simulation)
    # List tasks/gateways in order of appearance
    elements = []
    for elem in root.iter():
        tag = elem.tag.split('}')[-1]
        if tag in ['task', 'exclusiveGateway', 'startEvent', 'endEvent']:
            name = elem.get('name')
            if name: 
                # Clean up newlines in names
                clean_name = name.replace('\n', ' ')
                elements.append(f"{tag} executes '{clean_name}'.")
    
    generated_text = " Then ".join(elements) + "."
    
    return doc_text.strip(), generated_text

# ==========================================
# 3. INPUT DATA (Your XML Files)
# ==========================================

# 1. Carry On Baggage (Source 45)
xml_carry_on = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_1">
  <bpmn:collaboration id="Collaboration_CarryOn">
    <bpmn:documentation>The process for handling carry-on baggage starts with the passenger arriving at the airport. First, the airline or airport staff checks the baggage for compliance regarding amount, weight, and dimensions. If the baggage is not compliant, the passenger is required to pay a fee via credit card. Subsequently, prohibited items such as valuables and batteries must be removed, and the baggage is transported in the aircraft hold. The passenger proceeds to Boarding and enters the cabin. The Flight is ready for departure. If the baggage is compliant, the process checks for high flight occupancy. If occupancy is high, the staff offers a free check-in option for the compliant baggage. The passenger can then decide whether to accept this offer. If accepted, the baggage is prepared for the hold after removing sensitive items. If the offer is declined, or if the flight occupancy is normal, the passenger proceeds to boarding and enters the cabin. Inside the cabin, the stowage method depends on the item size. Large items are stowed in the overhead bins, while smaller personal items are placed under the seat in front. The process ends when the passenger is seated and ready for departure.</bpmn:documentation>
    <bpmn:participant id="Participant_Process" name="Carry-on Baggage Process" />
  </bpmn:collaboration>
  <bpmn:process id="Process_CarryOn_Refined" isExecutable="false">
    <bpmn:task id="Task_CheckCompliance" name="Check amount, weight and dimensions" />
    <bpmn:exclusiveGateway id="Gateway_Compliant" name="Baggage compliant?" />
    <bpmn:task id="Task_PayFee" name="Pay fee (Credit Card only)" />
    <bpmn:exclusiveGateway id="Gateway_Occupancy" name="High flight occupancy?" />
    <bpmn:task id="Task_OfferFreeCheckIn" name="Offer free check-in" />
    <bpmn:task id="Task_RemoveItems" name="Remove valuables &amp; batteries" />
    <bpmn:task id="Task_LoadHold" name="Transport in aircraft hold" />
    <bpmn:task id="Task_Boarding" name="Proceed to Boarding / Enter Cabin" />
    <bpmn:exclusiveGateway id="Gateway_ItemSize" name="Item size?" />
    <bpmn:task id="Task_StowOverhead" name="Stow in overhead bin" />
    <bpmn:task id="Task_StowUnderSeat" name="Stow under seat in front" />
    <bpmn:endEvent id="EndEvent_Seated" name="Ready for departure" />
    <bpmn:exclusiveGateway id="Gateway_AcceptFreeCheckIn" name="Passenger accepts option?" />
    <bpmn:startEvent id="StartEvent_1" name="Passenger arrives at Airport" />
  </bpmn:process>
</bpmn:definitions>"""

# 2. Passenger Arrival (Source 94)
xml_arrival = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_1d2hcmz">
  <bpmn:process id="Process_0gjrx3e" isExecutable="true">
    <bpmn:documentation>The process begins when the passenger arrives at the terminal. The flow immediately reaches an exclusive gateway that checks if the arrival is from the Schengen area. If the arrival is from the Schengen area, the passenger proceeds to passport control and shows their passport. If the arrival is not from the Schengen area, these control steps are skipped. Subsequently, the passenger goes to the baggage claim area. At this point, another gateway determines whether the passenger has baggage. If the passenger has baggage, they execute the task of taking their baggage. If they do not have baggage, they proceed directly to the next stage without taking baggage. Finally, the process reaches a decision point regarding customs. It checks if there are goods to declare. If there are goods to declare, the passenger goes to customs and declares them. If there are no goods to declare, the customs section is bypassed. The process concludes when the passenger leaves the airport.</bpmn:documentation>
    <bpmn:startEvent id="StartEvent_1" />
    <bpmn:task id="Activity_1mqzrms" name="Passenger arrives at terminal" />
    <bpmn:exclusiveGateway id="Gateway_0evfyb5" name="Arrival from Schengen area" />
    <bpmn:task id="Activity_025pe4c" name="Passenger goes to passport control" />
    <bpmn:task id="Activity_1os4btc" name="Passenger shows passport" />
    <bpmn:task id="Activity_08ly2sm" name="Passenger goes to Baggage claim" />
    <bpmn:exclusiveGateway id="Gateway_028unw1" name="Passenger has Baggage?" />
    <bpmn:task id="Activity_1ra6afl" name="Passenger takes baggage" />
    <bpmn:exclusiveGateway id="Gateway_1jm2qfx" name="Goods to declare?" />
    <bpmn:task id="Activity_1whqgsx" name="Passenger goes to customs" />
    <bpmn:task id="Activity_1uqy9wy" name="Passenger declares goods at customs" />
    <bpmn:task id="Activity_1ofe5a5" name="Passenger leaves airport" />
  </bpmn:process>
</bpmn:definitions>"""

# 3. Passenger Security (Source 125)
# NOTE: This file had NO documentation tag in the source. 
# We INJECT the manual text into <bpmn:documentation> so the pipeline works.
xml_security = """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" id="Definitions_Security">
  <bpmn:process id="Process_Security" isExecutable="true">
    <bpmn:documentation>The process is initiated when the passenger shows their boarding pass. Afterwards, the passenger proceeds to the security check area to undergo a security check. Following the screening, a decision is made regarding whether the passenger appears suspicious. If the passenger is deemed suspicious, they must undergo a manual control; if not, the manual control is skipped. Subsequently, the process evaluates the passenger's destination via a gateway. If the destination is outside the Schengen Area, the passenger goes to passport control and shows their passport. Conversely, if the destination is within the Schengen Area, the passport control steps are bypassed. Finally, the passenger goes to the gate, concluding the process.</bpmn:documentation>
    <bpmn:task name="Passenger shows boarding pass" />
    <bpmn:task name="Passenger goes to security check" />
    <bpmn:task name="Passenger is checked" />
    <bpmn:exclusiveGateway name="Suspicious?" />
    <bpmn:task name="manual control" />
    <bpmn:exclusiveGateway name="Target destination in Schengen-Area?" />
    <bpmn:task name="Passenger goes to passport control" />
    <bpmn:task name="Passenger shows passport" />
    <bpmn:task name="Passenger goes to gate" />
  </bpmn:process>
</bpmn:definitions>"""

# ==========================================
# 4. RUN EXPERIMENT
# ==========================================
experiments = [
    {"name": "Carry-On Baggage", "xml": xml_carry_on},
    {"name": "Passenger Arrival", "xml": xml_arrival},
    {"name": "Passenger Security", "xml": xml_security}
]

print(f"{'BPMN MODEL COHERENCE ANALYSIS':<40}")
print("=" * 80)

for exp in experiments:
    human_text, generated_text = parse_bpmn_data(exp["xml"])
    
    # Get Scores
    h_score, h_reason = get_coherence_score(human_text)
    g_score, g_reason = get_coherence_score(generated_text)
    
    # Print Results
    print(f"FILE: {exp['name']}")
    print("-" * 40)
    
    print(f"📘 GROUND TRUTH (Human)      [Score: {h_score}/10]")
    print(f"   Reason: {h_reason}")
    print(f"   Text: \"{human_text[:300]}...\"") # Truncated for display
    print("")
    
    print(f"🤖 GENERATED (Robotic)       [Score: {g_score}/10]")
    print(f"   Reason: {g_reason}")
    print(f"   Text: \"{generated_text[:300]}...\"") # Truncated for display
    
    print("=" * 80 + "\n")

BPMN MODEL COHERENCE ANALYSIS           
FILE: Carry-On Baggage
----------------------------------------
📘 GROUND TRUTH (Human)      [Score: 8/10]
   Reason: The text has a clear cause-and-effect structure, uses transition words (e.g., 'subsequently'), and maintains a logical flow. However, some sentences are slightly repetitive or could be rephrased for better clarity.
   Text: "The process for handling carry-on baggage starts with the passenger arriving at the airport. First, the airline or airport staff checks the baggage for compliance regarding amount, weight, and dimensions. If the baggage is not compliant, the passenger is required to pay a fee via credit card. Subseq..."

🤖 GENERATED (Robotic)       [Score: 5/10]
   Reason: The process description is clear and follows the correct sequence, but it lacks transition words and has a somewhat robotic tone.
   Text: "task executes 'Check amount, weight and dimensions'. Then exclusiveGateway executes 'Baggage compliant?'. Then task ex